In [48]:
import pandas as pd
from sqlalchemy import create_engine

In [49]:
# upload data
oscars_file = "Resources/the_oscar_award.csv"

# create dataframe and view data
raw_oscars_df = pd.read_csv(oscars_file)
raw_oscars_df.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [69]:
# filter dataframe for winners and rename name to name of winner
oscars_data_df = raw_oscars_df.loc[raw_oscars_df['winner'] == True].rename(columns={'name': 'name_of_winner'}).dropna()

# set index to film for upload in SQL as primary key
oscars_data_df.set_index(['film'], inplace = True)

oscars_data_df


,year_film,year_ceremony,ceremony,category,name_of_winner,winner
film,,,,,,
The Last Command,1927,1928,1,ACTOR,Emil Jannings,True
7th Heaven,1927,1928,1,ACTRESS,Janet Gaynor,True
The Dove;,1927,1928,1,ART DIRECTION,William Cameron Menzies,True
Sunrise,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,True
Sunrise,1927,1928,1,CINEMATOGRAPHY,Karl Struss,True
...,...,...,...,...,...,...
Ford v Ferrari,2019,2020,92,SOUND EDITING,Donald Sylvester,True
1917,2019,2020,92,SOUND MIXING,Mark Taylor and Stuart Wilson,True
1917,2019,2020,92,VISUAL EFFECTS,"Guillaume Rocheron, Greg Butler and Dominic Tuohy",True


In [70]:
# create a separate dataframe that will summarize the number of awards by film 
subset_oscars = oscars_data_df[['category']]

subset_oscars

,category
film,
The Last Command,ACTOR
7th Heaven,ACTRESS
The Dove;,ART DIRECTION
Sunrise,CINEMATOGRAPHY
Sunrise,CINEMATOGRAPHY
...,...
Ford v Ferrari,SOUND EDITING
1917,SOUND MIXING
1917,VISUAL EFFECTS


In [71]:
# count categories won by film and sort descending
oscars_by_film =subset_oscars.groupby('film').count().sort_values('category', ascending = False).rename(columns={'category': 'categories_won'})

oscars_by_film.head(30)

,categories_won
film,
Titanic,12
The Lord of the Rings: The Return of the King,11
Ben-Hur,11
West Side Story,10
Gigi,9
The English Patient,9
The Last Emperor,9
Gandhi,8
On the Waterfront,8


In [72]:
# import pgAdmin password 
from my_secrets import pw

In [73]:
# create connection to the database and the engine
rds_connection_string = f"postgres:{pw}@localhost:5432/movies_db"
engine = create_engine(f'postgresql://{rds_connection_string}')



In [75]:
# upload dataframes to SQL database
oscars_data_df.to_sql(name='oscars_data', con=engine, if_exists='append', index=True)
oscars_by_film.to_sql(name='total_oscars_by_film', con=engine, if_exists='append', index=True)

In [76]:
# pull table names to test
engine.table_names()

['imdb', 'oscars_data', 'total_oscars_by_film']